In [9]:
import pandas as pd
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
from typing import List, Dict, Tuple, Any, Type, Union
import ast

In [10]:
import election_project as ep

In [11]:
df = pd.read_csv('data/final_dataset.csv')

In [14]:
indices = np.where(df['year'] == 2020)[0]
indices

array([ 9270,  9271,  9272, ..., 12357, 12358, 12359], shape=(3090,))

In [19]:
df[df['year']==2020]

,year,gisjoin,state,county,median_household_income,per_capita_income,per_capita_area,per_capita_households_income_under_10k,per_capita_households_income_10k_15k,per_capita_households_income_15k_25k,...,P(labor_force_employed|C),P(labor_force_unemployed|C),P(not_in_labor_force|C),P(persons_hispanic|C),P(persons_below_poverty|C),P(18plus|C),P(democrat|C),P(other|C),P(republican|C),P(non_voter|C)
9270,2020,G0100010,Alabama,Autauga County,68315,35332,0.026639,0.016848,0.011164,0.040180,...,0.449465,0.012798,0.324024,0.031722,0.112830,0.765729,0.127687,0.007301,0.337605,0.293137
9271,2020,G0100030,Alabama,Baldwin County,71039,38907,0.022494,0.017351,0.016370,0.027281,...,0.459489,0.016387,0.333318,0.048025,0.100441,0.788381,0.105295,0.006670,0.357913,0.318503
9272,2020,G0100050,Alabama,Barbour County,39712,23378,0.094171,0.036861,0.029787,0.059010,...,0.339752,0.020742,0.459501,0.048318,0.212244,0.792941,0.193592,0.003216,0.225992,0.370141
9273,2020,G0100070,Alabama,Bibb County,50669,24966,0.072885,0.024673,0.029662,0.035459,...,0.370410,0.035324,0.405285,0.029212,0.193115,0.788099,0.089254,0.003775,0.338187,0.356883
9274,2020,G0100090,Alabama,Blount County,57440,29053,0.028524,0.024460,0.018992,0.030943,...,0.420739,0.026711,0.352117,0.096840,0.140105,0.770401,0.044687,0.004012,0.418285,0.303418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12355,2020,G5600370,Wyoming,Sweetwater County,79375,40268,0.645738,0.020248,0.011003,0.025904,...,0.497873,0.030158,0.245063,0.162076,0.114855,0.745146,0.090853,0.015352,0.290620,0.348321
12356,2020,G5600390,Wyoming,Teton County,108279,76296,0.467860,0.007753,0.008652,0.024372,...,0.649190,0.013878,0.180245,0.151975,0.068791,0.823739,0.421828,0.025615,0.185942,0.190354
12357,2020,G5600410,Wyoming,Uinta County,78164,32955,0.263227,0.007495,0.012216,0.019420,...,0.465395,0.016500,0.271294,0.098559,0.064003,0.720724,0.077436,0.018106,0.364840,0.260343
12358,2020,G5600430,Wyoming,Washakie County,61875,32979,0.751918,0.016570,0.014239,0.047249,...,0.512751,0.009320,0.293204,0.142783,0.065113,0.781230,0.084272,0.017605,0.420065,0.259288


In [20]:
indices = (df[df['year'] == 2020]).index.tolist()
indices

[9270,
 9271,
 9272,
 9273,
 9274,
 9275,
 9276,
 9277,
 9278,
 9279,
 9280,
 9281,
 9282,
 9283,
 9284,
 9285,
 9286,
 9287,
 9288,
 9289,
 9290,
 9291,
 9292,
 9293,
 9294,
 9295,
 9296,
 9297,
 9298,
 9299,
 9300,
 9301,
 9302,
 9303,
 9304,
 9305,
 9306,
 9307,
 9308,
 9309,
 9310,
 9311,
 9312,
 9313,
 9314,
 9315,
 9316,
 9317,
 9318,
 9319,
 9320,
 9321,
 9322,
 9323,
 9324,
 9325,
 9326,
 9327,
 9328,
 9329,
 9330,
 9331,
 9332,
 9333,
 9334,
 9335,
 9336,
 9337,
 9338,
 9339,
 9340,
 9341,
 9342,
 9343,
 9344,
 9345,
 9346,
 9347,
 9348,
 9349,
 9350,
 9351,
 9352,
 9353,
 9354,
 9355,
 9356,
 9357,
 9358,
 9359,
 9360,
 9361,
 9362,
 9363,
 9364,
 9365,
 9366,
 9367,
 9368,
 9369,
 9370,
 9371,
 9372,
 9373,
 9374,
 9375,
 9376,
 9377,
 9378,
 9379,
 9380,
 9381,
 9382,
 9383,
 9384,
 9385,
 9386,
 9387,
 9388,
 9389,
 9390,
 9391,
 9392,
 9393,
 9394,
 9395,
 9396,
 9397,
 9398,
 9399,
 9400,
 9401,
 9402,
 9403,
 9404,
 9405,
 9406,
 9407,
 9408,
 9409,
 9410,
 9411,
 9412,

In [ ]:
# --- Default Hyperparameter Grids for CV ---
RIDGE_PARAM_GRID = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]

MLP1_PARAM_GRID = {
    'n_hidden': [16, 64, 128],
    'dropout_rate': [0.1, 0.3, 0.5],
    'learning_rate': [1e-2, 1e-3, 1e-4]
    # Note: weight_decay could be added here too if desired
}
MLP2_PARAM_GRID = {
    'shared_hidden_size': [16, 32, 64],
    'dropout_rate': [0.1, 0.3, 0.5],
    'learning_rate': [1e-2, 1e-3, 1e-4]
    # Note: weight_decay could be added here too if desired
}

# --- XGBoost Hyperparameter Grid and Constants ---
XGB_PARAM_GRID = {
    'learning_rate': [0.05, 0.1, 0.2],     # Step size shrinkage (eta)
    'max_depth': [5, 7],                # Max depth of a tree
    'subsample': [0.8, 1.0],         # Fraction of samples used per tree
    'colsample_bytree': [0.8, 1.0],  # Fraction of features used per tree
    'gamma': [0.1, 0.2],                # Min loss reduction for split (min_split_loss)
    'reg_alpha': [0, 0.1, 1.0],            # L1 regularization
    'reg_lambda': [0, 0.1, 1.0],           # L2 regularization
    # Fixed parameters for consistency
    'objective': ['reg:squarederror'], # Regression objective for each target
    'n_estimators': [200],             # High initial value, CV uses early stopping
    'random_state': [42]               # For reproducibility
}

XGB_EARLY_STOPPING_ROUNDS = 20 # Early stopping rounds for CV fits

RUNG_EPOCHS = [25, 50, 75, 100, 125, 150, 175, 200] # Rung epochs for MLP models
RUNG_PATIENCE = [15, 20, 25, 30, 35, 40, 45, 50] # Rung patience for MLP models

In [4]:
#set a manual seed for reproducibility
torch.manual_seed(42)
dh = ep.DataHandler()
dh.update_cv_dataloaders(batch_size=BATCH_SIZE)
dh.update_final_dataloader(batch_size=BATCH_SIZE)

DataHandler initialized - Using 114 features - Test year: 2020
Updated cross-validation DataLoaders with batch size 64.
Updated final training DataLoader with batch size 64.


In [5]:
MLP1_PARAM_GRID = {
    'hidden_layers': [[16], [32], [64], [128]],       # Width of the single hidden layer
    'learning_rate': [1e-2, 1e-3, 1e-4],        # Optimizer learning rate
    'dropout_rate': [0.0, 0.1, 0.2],           # Dropout regularization
    'weight_decay': [0, 1e-5, 1e-3]             # L2 regularization (AdamW style)
}

RUNG_EPOCHS = [25, 50, 75, 100, 125, 150, 175, 200] # Rung epochs for MLP models
RUNG_PATIENCE = [15, 20, 25, 30, 35, 40, 45, 50] # Rung patience for MLP models
MLP1_SCHEDULE = list(zip(RUNG_EPOCHS, RUNG_PATIENCE))

#make mlp1 model
mlp1_model = ep.NNModel(model_name = 'mlp1')

NNModel initialized for 'mlp1'. CV results: ./results/mlp1_cv_results.csv


In [6]:
mlp1_model._parse_best_params_from_csv()

{'dropout_rate': 0,
 'hidden_layers': [128],
 'learning_rate': np.float64(0.001),
 'weight_decay': np.float64(1e-05),
 'last_epoch': 153,
 'best_epoch': 104,
 'train_loss_at_best': np.float64(0.832217),
 'best_val_loss': np.float64(0.839444)}

In [7]:
mlp1_model.train_final_model(dh, 
                             final_train_epochs=FINAL_TRAIN_EPOCHS,
                             optimizer_choice=OPTIMIZER_CHOICE)


--- Starting Final Model Training for MLP1 ---
Using best hyperparameters from CV: {'dropout_rate': 0, 'hidden_layers': [128], 'learning_rate': np.float64(0.001), 'weight_decay': np.float64(1e-05), 'last_epoch': 153, 'best_epoch': 104, 'train_loss_at_best': np.float64(0.832217), 'best_val_loss': np.float64(0.839444)}
Starting final training for up to 150 epochs (Patience: 50)...
  Epoch 10/150 - Loss: 0.841157 (Best Loss: 0.841157, Epochs No Improve: 0)
  Epoch 20/150 - Loss: 0.838287 (Best Loss: 0.838287, Epochs No Improve: 0)
  Epoch 30/150 - Loss: 0.838319 (Best Loss: 0.837144, Epochs No Improve: 2)
  Epoch 40/150 - Loss: 0.838080 (Best Loss: 0.835730, Epochs No Improve: 1)
  Epoch 50/150 - Loss: 0.835654 (Best Loss: 0.835585, Epochs No Improve: 1)
  Epoch 60/150 - Loss: 0.835354 (Best Loss: 0.835354, Epochs No Improve: 0)
  Epoch 70/150 - Loss: 0.835682 (Best Loss: 0.834135, Epochs No Improve: 6)
  Epoch 80/150 - Loss: 0.834687 (Best Loss: 0.834135, Epochs No Improve: 16)
  Epoch 

Sequential(
  (0): Linear(in_features=114, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0, inplace=False)
  (3): Linear(in_features=128, out_features=4, bias=True)
  (4): Softmax(dim=1)
)

In [8]:
mlp1_model.predict(dh, save=True)


--- Generating Predictions for MLP1 on Year 2020 ---
County-level raw predictions saved to: ./preds/2020_mlp1_predictions.csv


array([[0.15219903, 0.02495648, 0.26912528, 0.31944814],
       [0.15105082, 0.03149771, 0.31526598, 0.29056692],
       [0.18049634, 0.0103337 , 0.20910275, 0.3930085 ],
       ...,
       [0.08355194, 0.05605646, 0.28210762, 0.29900822],
       [0.09211148, 0.04730706, 0.36937997, 0.27243122],
       [0.11738477, 0.03578475, 0.27462345, 0.3679858 ]],
      shape=(3090, 4), dtype=float32)

In [11]:
mlp1_model.cross_validate(dh,
                          param_grid=MLP1_PARAM_GRID,
                          optimizer_choice=OPTIMIZER_CHOICE,
                          rung_schedule=MLP1_SCHEDULE
                          )



--- Starting SHA Cross-Validation for MLP1 (eta=3) ---
-------------------------------------------------------------------------------
>>> SHA Rung 1/8 | Target Epochs: 25 | Patience: 15 | Evaluating 108 configs <<<
-------------------------------------------------------------------------------
| Config | Last Epoch | Best Epoch | Train Loss |  Val Loss  | Time (seconds) |
|     1  |     23     |     15     |  0.838874  |  0.845222  |      27.59s    |
|     2  |     22     |     11     |  0.841238  |  0.843598  |      25.90s    |
|     3  |     25     |     19     |  0.840484  |  0.845393  |      29.50s    |
|     4  |     25     |     21     |  0.838757  |  0.845708  |      29.60s    |
|     5  |     25     |     19     |  0.838702  |  0.845632  |      29.88s    |
|     6  |     25     |     20     |  0.839764  |  0.846116  |      29.90s    |
|     7  |     25     |     24     |  0.844878  |  0.853489  |      30.43s    |
|     8  |     25     |     24     |  0.846943  |  0.854509  | 

KeyboardInterrupt: 

In [5]:
#make softmax model
softmax_model = ep.NNModel(model_name = 'Softmax')

# param grid and rung schedule for softmax model
SOFTMAX_PARAM_GRID = {'learning_rate': [1e-1, 1e-2, 1e-3, 1e-4],
                      'weight_decay': [0, 1e-5, 1e-4, 1e-3]
                    }
SOFTMAX_SCHEDULE = [(50, 20), (75, 25), (100, 30), (150, 40)]

NNModel initialized for 'Softmax'. CV results: ./results/Softmax_cv_results.csv


In [6]:
softmax_model.cross_validate(dh,
                             param_grid=SOFTMAX_PARAM_GRID,
                             optimizer_choice=OPTIMIZER_CHOICE,
                             rung_schedule=SOFTMAX_SCHEDULE
                             )


--- Starting SHA Cross-Validation for SOFTMAX (eta=3) ---
-------------------------------------------------------------------------------

>>> SHA Rung 1/4 | Target Epochs: 50 | Patience: 20 | Evaluating 16 configs <<<
-------------------------------------------------------------------------------
| Config | Last Epoch | Best Epoch | Train Loss |  Val Loss  | Time (seconds) |
|     1  |     38     |     21     |   1.09341  |   1.05917  |      43.09s    |
|     2  |     46     |     31     |  1.083709  |  1.058134  |      49.63s    |
|     3  |     22     |      2     |  1.229732  |  1.062608  |      23.81s    |
|     4  |     23     |      3     |  1.147415  |  1.058997  |      25.25s    |
|     5  |     38     |     18     |  0.841234  |  0.844526  |      41.64s    |
|     6  |     39     |     19     |   0.84157  |  0.843677  |      42.12s    |
|     7  |     38     |     19     |  0.841895  |  0.844437  |      41.00s    |
|     8  |     39     |     22     |  0.841344  |  0.844243 

,learning_rate,weight_decay,last_epoch,best_epoch,train_loss_at_best,best_val_loss
0,0.01,0.00100,87,47,0.842698,0.843331
1,0.01,0.00010,92,53,0.840768,0.843501
2,0.01,0.00001,59,19,0.841570,0.843677


In [13]:
softmax_model.train_final_model(dh,
                                final_train_epochs= FINAL_TRAIN_EPOCHS,
                                optimizer_choice=OPTIMIZER_CHOICE
                                )


--- Starting Final Model Training for SOFTMAX ---
Using best hyperparameters from CV: {'learning_rate': 0.01, 'weight_decay': 0.0001, 'last_epoch': 77.66666666666667, 'best_epoch': 37.666666666666664, 'train_loss_at_best': 0.8412514337149682, 'best_val_loss': 0.8426987974822117}
Starting final training for up to 150 epochs (Patience: 50)...
  Epoch 10/150 - Loss: 0.843121 (Best Loss: 0.843121, Epochs No Improve: 0)
  Epoch 20/150 - Loss: 0.841399 (Best Loss: 0.841399, Epochs No Improve: 0)
  Epoch 30/150 - Loss: 0.841244 (Best Loss: 0.840897, Epochs No Improve: 3)
  Epoch 40/150 - Loss: 0.840943 (Best Loss: 0.840566, Epochs No Improve: 9)
  Epoch 50/150 - Loss: 0.843518 (Best Loss: 0.839810, Epochs No Improve: 1)
  Epoch 60/150 - Loss: 0.840637 (Best Loss: 0.839810, Epochs No Improve: 11)
  Epoch 70/150 - Loss: 0.841734 (Best Loss: 0.839807, Epochs No Improve: 1)
  Epoch 80/150 - Loss: 0.840093 (Best Loss: 0.839807, Epochs No Improve: 11)
  Epoch 90/150 - Loss: 0.841632 (Best Loss: 0.

Sequential(
  (0): Linear(in_features=114, out_features=4, bias=True)
  (1): Softmax(dim=1)
)

In [14]:
softmax_preds = softmax_model.predict(dh, save=True)


--- Generating Predictions for SOFTMAX on Year 2020 ---
County-level raw predictions saved to: ./preds/2020_Softmax_predictions.csv


In [15]:
softmax_preds

array([[0.15207319, 0.02530907, 0.2590983 , 0.32924837],
       [0.15103795, 0.06097275, 0.2786021 , 0.29776868],
       [0.20952702, 0.0151517 , 0.16357335, 0.4046892 ],
       ...,
       [0.11637519, 0.05959414, 0.2536456 , 0.2911093 ],
       [0.15302789, 0.04937229, 0.29364637, 0.2851832 ],
       [0.08838788, 0.12065135, 0.21881565, 0.3679239 ]],
      shape=(3090, 4), dtype=float32)